In [2]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
import math
from variable_functions import *

In [3]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [4]:
with open('../../../filelists/800to1000_files.txt', 'r') as f:
    files = [line.strip() for line in f]

In [4]:
files[10]

'nano_mc2017_1-108.root'

In [18]:
events = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/qcd/800to1000/' + files[10]: "/Events"},
    delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [19]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [22]:
fatjetSelect = (
            (events.FatJet.pt > 400)
            & (events.FatJet.pt < 800)
            #& (events.FatJet.num_subjets >= 3)
            & (abs(events.FatJet.eta) < 2.5)
            & (events.FatJet.msoftdrop > 90)
            & (events.FatJet.msoftdrop < 150)
        )
# events = uncut_events[
#     ak.any(fatjetSelect, axis=1)
# ]

events["goodjets"] = events.FatJet[fatjetSelect]

# cut = (ak.num(uncut_events.FatJet.constituents.pf, axis=2) > 0)
# events = uncut_events[ak.all(cut, axis=1)]

In [25]:
ak.where(ak.flatten(events.goodjets.pt) > 800)

(<Array [] type='0 * int64'>,)

In [148]:
events['FatJet', 'd2b1'] = ak.unflatten(
     d2_calc(events.FatJet), counts=ak.num(events.FatJet)
)

In [9]:
events.FatJet.fields

['area',
 'btagCSVV2',
 'btagDDBvLV2',
 'btagDDCvBV2',
 'btagDDCvLV2',
 'btagDeepB',
 'btagHbb',
 'deepTagMD_H4qvsQCD',
 'deepTagMD_HbbvsQCD',
 'deepTagMD_TvsQCD',
 'deepTagMD_WvsQCD',
 'deepTagMD_ZHbbvsQCD',
 'deepTagMD_ZHccvsQCD',
 'deepTagMD_ZbbvsQCD',
 'deepTagMD_ZvsQCD',
 'deepTagMD_bbvsLight',
 'deepTagMD_ccvsLight',
 'deepTag_H',
 'deepTag_QCD',
 'deepTag_QCDothers',
 'deepTag_TvsQCD',
 'deepTag_WvsQCD',
 'deepTag_ZvsQCD',
 'eta',
 'mass',
 'msoftdrop',
 'n2b1',
 'n3b1',
 'particleNetMD_QCD',
 'particleNetMD_Xbb',
 'particleNetMD_Xcc',
 'particleNetMD_Xqq',
 'particleNet_H4qvsQCD',
 'particleNet_HbbvsQCD',
 'particleNet_HccvsQCD',
 'particleNet_QCD',
 'particleNet_TvsQCD',
 'particleNet_WvsQCD',
 'particleNet_ZvsQCD',
 'particleNet_mass',
 'phi',
 'pt',
 'rawFactor',
 'tau1',
 'tau2',
 'tau3',
 'tau4',
 'lsf3',
 'jetId',
 'subJetIdx1',
 'subJetIdx2',
 'electronIdx3SJ',
 'muonIdx3SJ',
 'nConstituents',
 'DDX_tau1_flightDistance2dSig',
 'DDX_tau1_trackEtaRel_0',
 'DDX_tau1_trackEt

In [28]:
import pandas as pd
a = pd.read_parquet('/project01/ndcms/cmoore24/skims/fatjet_skims/hgg', engine='fastparquet')

In [29]:
a.keys()

Index(['FatJets.area', 'FatJets.btagCSVV2', 'FatJets.btagDDBvLV2',
       'FatJets.btagDDCvBV2', 'FatJets.btagDDCvLV2', 'FatJets.btagDeepB',
       'FatJets.btagHbb', 'FatJets.deepTagMD_H4qvsQCD',
       'FatJets.deepTagMD_HbbvsQCD', 'FatJets.deepTagMD_TvsQCD',
       'FatJets.deepTagMD_WvsQCD', 'FatJets.deepTagMD_ZHbbvsQCD',
       'FatJets.deepTagMD_ZHccvsQCD', 'FatJets.deepTagMD_ZbbvsQCD',
       'FatJets.deepTagMD_ZvsQCD', 'FatJets.deepTagMD_bbvsLight',
       'FatJets.deepTagMD_ccvsLight', 'FatJets.deepTag_H',
       'FatJets.deepTag_QCD', 'FatJets.deepTag_QCDothers',
       'FatJets.deepTag_TvsQCD', 'FatJets.deepTag_WvsQCD',
       'FatJets.deepTag_ZvsQCD', 'FatJets.eta', 'FatJets.mass',
       'FatJets.msoftdrop', 'FatJets.n2b1', 'FatJets.n3b1',
       'FatJets.particleNetMD_QCD', 'FatJets.particleNetMD_Xbb',
       'FatJets.particleNetMD_Xcc', 'FatJets.particleNetMD_Xqq',
       'FatJets.particleNet_H4qvsQCD', 'FatJets.particleNet_HbbvsQCD',
       'FatJets.particleNet_HccvsQCD